In [ ]:
# conda install -c conda-forge pyomo
# conda install -c conda-forge pyomo.extras
# conda install -c conda-forge glpk
# conda install -c conda-forge ipopt
#
# Optionally, you may also wish to install the COIN-OR cbc solvers.
#
# conda install -c conda-forge coincbc
# conda install -c conda-forge cvxpy

# https://www.coin-or.org/download/binary/Ipopt/   # iptopt can be found here .. copy the executable and extract it at appropriate location

# have to download trial license key for mosek ## not needed

In [56]:
import math

import pyomo.core
import pyomo.environ as pyo
from pyomo.environ import *
import cvxpy as cp
import numpy as np


\begin{aligned}
\min_{x,y} \quad & 12x + 20y \\
\textrm{s.t.} \quad & x \geq 0\\

  & 0 \leq y \leq 3  \\
  & 6x + 8y \geq 100 \\
  & 7x + 12y \geq 120 \\

\end{aligned}

In [77]:
# cp.installed_solvers()

In [68]:

c = np.array([12,20])
A = np.array([ [-1, 0], [0, -1], [0, 1], [-6, -8], [-7, -12]])
b = np.array([0, 0, 3, -100, -120])

x = cp.Variable(len(c))
prob = cp.Problem(cp.Minimize(c.T@x), [A @ x <= b])
prob.solve(solver="ECOS")

print(f"Optimal solution ::  x = {prob.value}")
print(f"Optimal value :: {x.value}")

# print("A dual solution is")
# print(prob.constraints[0].dual_value)

Optimal solution ::  x = 204.9999999934052
Optimal value :: [15.00000011  1.24999993]


In [61]:
# Using Pyomo

model = ConcreteModel()

model.x = Var(domain=pyo.NonNegativeReals)
model.y = Var(bounds=(0,3))


model.objective = Objective(expr = 12*model.x + 20*model.y)

model.constraints = ConstraintList()
model.constraints.add(expr = 6*model.x + 8*model.y >= 100)
model.constraints.add(expr = 7*model.x + 12*model.y >= 120)


results = SolverFactory('glpk').solve(model)
# results.write()
# if results.solver.status:
#     model.pprint()

if results.solver.status:
    condition = results.solver[0]["Termination condition"]
    print(f"Termination condition :: {condition}")
    print(f"Optimal solution ::  x = {model.x()}, y = {model.y()}")
    print(f"Optimal value :: {model.objective()}")
    print("Solved!!")
else:
    print("Not solved!")


Termination condition :: optimal
Optimal solution ::  x = 15.0, y = 1.25
Optimal value :: 205.0
Solved!!


\begin{aligned}
\min_{x} \quad & x/8 + \sqrt{(6-x)^2 + 10x^3 } \\
\textrm{s.t.} \quad & 0 \leq x \leq 6  \\

\end{aligned}

In [51]:
model = pyo.ConcreteModel()

# declare decision variables
model.x = pyo.Var(initialize=0.0, bounds=(0,6))
# declare objective
model.objective = pyo.Objective(
    expr = model.x/8 + pyomo.core.sqrt((6.0-model.x**2) + 10.0*model.x**3),
    sense = pyo.minimize)

# solve
results = pyo.SolverFactory('ipopt',executable="D:\\cli\\ipopt.exe").solve(model)

# results.write()
# if results.solver.status == 'ok':
#     model.pprint()

if results.solver.status:
    condition = results.solver[0]["Termination condition"]
    print(f"Termination condition :: {condition}")
    print(f"Optimal solution ::  x = {model.x()}")
    print(f"Optimal value :: {model.objective()}")
    print("Solved!!")
else:
    print("Not solved!")

Termination condition :: optimal
Optimal solution ::  x = 8.974906424446161e-09
Optimal value :: 2.4494897439050414
Solved!!


\begin{aligned}
\min_{x} \quad & x^\top Q x \\
\textrm{s.t.} \quad & x \geq 0\\

  & \sum_{i=1}^{n} x_i \leq 1000\\
  & r^\top x \geq 50 \\
\end{aligned}

In [76]:
# Generate data.

r = np.array([0.0260022, 0.00810132, 0.0737159])
Q = np.array([[ 0.018641, 0.00359853, 0.00130976],
              [0.00359853, 0.00643694,0.00488727],
              [0.00130976, 0.00488727, 0.0686828]])

# Define and solve the CVXPY problem.
x = cp.Variable(Q.shape[0])
prob = cp.Problem(cp.Minimize(cp.quad_form(x,Q)), [x >= 0, sum(x) <= 1000, r.T @ x >= 50] )

prob.solve(solver="ECOS")

print(f"Optimal solution ::  x = {prob.value}")
print(f"Optimal value :: {x.value}")

Optimal solution ::  x = 22635.102877893845
Optimal value :: [4.96935835e+02 3.52796825e-02 5.02990893e+02]


\begin{aligned}
\min_{x} \quad & ||Ax-b||^2 \\
\textrm{s.t.} \quad & x \geq 0\\
\end{aligned}

In [57]:

# Generate data.
m = 20
n = 15
np.random.seed(1)
A = np.random.randn(m, n)
b = np.random.randn(m)

# Define and solve the CVXPY problem.
x = cp.Variable(n)
cost = cp.sum_squares(A @ x - b)
prob = cp.Problem(cp.Minimize(cost))
prob.solve()

# Print result.
print("\nThe optimal value is", prob.value)
print("The optimal x is")
print(x.value)
print("The norm of the residual is ", cp.norm(A @ x - b, p=2).value)


The optimal value is 7.005909828287486
The optimal x is
[ 0.17492418 -0.38102551  0.34732251  0.0173098  -0.0845784  -0.08134019
  0.293119    0.27019762  0.17493179 -0.23953449  0.64097935 -0.41633637
  0.12799688  0.1063942  -0.32158411]
The norm of the residual is  2.6468679280023557
